In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import pymc as pm
%matplotlib inline

# TODO: make ceam-public-health available enough to replicate model results

Abie Flaxman [4:25 PM] 
@emumford : how do you launch your diarrhea model on the cluster?

Everett Mumford [4:32 PM] 
simulate run ceam-public-health/configurations/rota_vaccine_intervention.json

[4:33] 
simulate run ceam-public-health/configurations/ors.json

[4:35] 
If you want to launch multiple scnarios and/or draws, you would run --> python ceam/scripts/rota_vaccine_cluster_runner.py -g (the -g is required if you are running multiple scenarios) for rota (edited)

[4:35] 
and for ors --> python ceam/scripts/ors_cluster_runner.py -g

Abie Flaxman [5:43 PM] 
if i do `python ceam/scripts/rota_vaccine_cluster_runner.py -g `, what will come out?

new messages
Everett Mumford [5:48 PM] 
The file on Stash looks like it is set up to run all 12 scenarios that we wanted to run before the BMGF meeting in Feb. for Kenya and Bangladesh. The default is to do all 1k draws for each scenario, but you can change this using the --runs arg

[5:49] 
I think this code is going to need to be changed a bit given the config file work that Alec merged in earlier today, but as long as you haven't merged develop into your branch yet it should work

Everett Mumford [5:55 PM] 
The results for each scenario should be sent to /share/scratch/users/{username}/results/{scenario parameters}/{timestamp of when the jobs were launched}/runs.hdf, though you can change the output directory using the --results_dir argument

Abie Flaxman [7:15 PM] 
Thanks Everett

In [2]:
df = pd.read_hdf('/homes/alecwd/raw_grid_results.hdf')
df.head()

,age_at_first_dose,comparison,cost_to_administer_rotaviral_entiritis_vaccine,deaths,diarrhea_due_to_EPEC_event_count,diarrhea_due_to_ETEC_event_count,diarrhea_due_to_adenovirus_event_count,diarrhea_due_to_aeromonas_event_count,diarrhea_due_to_amoebiasis_event_count,diarrhea_due_to_campylobacter_event_count,...,rotaviral_entiritis_vaccine_unit_cost,run_intervention,simulation_run_time,total_population,total_population__dead,total_population__living,vaccination_proportion_increase,vaccine_duration,years_lived_with_disability,years_of_life_lost
0,7,base,0.0,973,1837.0,5162.0,3268.0,10607.0,1016.0,7164.0,...,50953.0,True,3407.383937,10000,973,9027,0.5,1000,928.339509,83716.682292
0,7,base,0.0,923,2793.0,6181.0,3443.0,6663.0,240.0,7437.0,...,51194.5,True,3794.690914,10000,923,9077,0.5,1000,993.516558,79400.318812
0,7,base,0.0,993,2103.0,4542.0,5057.0,5051.0,960.0,3966.0,...,50697.5,True,4575.946919,10000,993,9007,0.5,1000,901.553985,85466.076431
0,7,base,0.0,1038,4261.0,5087.0,4261.0,15043.0,2141.0,3464.0,...,50806.0,True,3761.741159,10000,1038,8962,0.5,1000,1084.532730,89292.691085
0,7,base,0.0,922,2131.0,4894.0,4282.0,7398.0,454.0,2491.0,...,51250.5,True,4654.364322,10000,922,9078,0.5,1000,807.743677,79307.723380


In [3]:
# need to select the scenarios of interest
location_id = 180

# set baseline scenario for Kenya
df.loc[(df.location == '{}'.format(location_id)) & (df.age_at_first_dose == '61') &
       (df.vaccine_duration == '730') & (df.vaccination_proportion_increase == '0'), 'comparison'] = 'baseline'

In [4]:
# set % coverage scenario
for coverage in [.25, .5 , .75]:
    df.loc[(df.location == '{}'.format(location_id)) & (df.age_at_first_dose == '61') &
           (df.vaccine_duration == '730') & (df.vaccination_proportion_increase == '{}'.format(coverage)), 'comparison'] = '{}% cov.'.format(int(float(coverage)*100))

In [5]:
df.comparison.value_counts()

base        17477
baseline      659
50% cov.      628
25% cov.      375
75% cov.      244
Name: comparison, dtype: int64

In [6]:
baseline = df.query('comparison == "baseline"').copy()
baseline['DALYs'] = baseline.years_lived_with_disability + baseline.years_of_life_lost

In [7]:
intervention = df.query('comparison == "50% cov."').copy()
intervention['DALYs'] = intervention.years_lived_with_disability + intervention.years_of_life_lost

In [8]:
t = pd.merge(baseline, intervention, on='draw', suffixes=('_baseline', '_intervention'))
t.filter(like='DALY').head()

,DALYs_baseline,DALYs_intervention
0,73485.125429,73205.599786
1,77456.666215,77260.018482
2,71830.059856,71285.085447
3,75837.502812,75643.298546
4,72417.504031,72308.580793


In [9]:
(t.DALYs_baseline - t.DALYs_intervention)\
    .describe(percentiles=[.025,.975]).loc[['mean', '2.5%', '97.5%']]

mean     385.874470
2.5%      34.092294
97.5%    808.516097
dtype: float64

In [10]:
np.random.seed(12345)

In [11]:
permuted_vals = np.array(t.DALYs_intervention)
np.random.shuffle(permuted_vals)

In [12]:
(t.DALYs_baseline - permuted_vals)\
    .describe(percentiles=[.025,.975]).loc[['mean', '2.5%', '97.5%']]

mean      385.874470
2.5%    -7455.514508
97.5%    8523.486522
Name: DALYs_baseline, dtype: float64

In [13]:
(8523 - -7455) / (808.5-34.09)

20.63248150204672